# Оперон PDU сравним с другими оперонами в том же наборе штаммов
***

# [21.09]

In [1]:
import pandas as pd

In [57]:
og = pd.read_csv("orthosnake/pdu/Results/Orthogroups.tsv", sep="\t", header = None)

In [18]:
core_og_pd =  pd.read_csv("orthosnake/pdu/Results/Orthogroups_SingleCopyOrthologues.txt", header = None)
core_og = core_og_pd[0].to_list()

In [43]:
core_og_pd = og[og[0].isin(core_og)]
core_og_in_genome = core_og_pd[1]
og_by_cds = dict(zip(core_og_in_genome, core_og_pd[0]))

#core_og_in_genome[0:3]

In [50]:
ids = [int(cds.split("|")[1]) for cds in core_og_in_genome]
og_by_id = {int(cds.split("|")[1]):og_by_cds[cds] for cds in core_og_in_genome}


In [68]:
from itertools import groupby
from operator import itemgetter
og_core_sets = []
for k, g in groupby(enumerate(ids), lambda ix : ix[0] - ix[1]):
    ids_seq = list(map(itemgetter(1), g))
    if(len(ids_seq) < 4): 
        continue
    og_core_sets.append([og_by_id[x] for x in ids_seq])
og_core_sets[0:3]

[['OG0000875', 'OG0000876', 'OG0000877', 'OG0000878', 'OG0000879'],
 ['OG0000885',
  'OG0000886',
  'OG0000887',
  'OG0000888',
  'OG0000889',
  'OG0000890',
  'OG0000891',
  'OG0000892'],
 ['OG0000893', 'OG0000894', 'OG0000895', 'OG0000896']]

### write to files

In [70]:
!mkdir -p tree_to_compare/
!mkdir -p tree_to_compare/pdu
!mkdir -p tree_to_compare/pdu/og_lists/`

In [ ]:
i = 0
for og_core_set in og_core_sets:
    fname = "tree_to_compare/pdu/og_lists/{}".format(i)
    i = i + 1
    with open(fname, 'w') as filehandle:
        for x in og_core_set:
            filehandle.write('%s\n' % x)


## [16.09]

### найдем опероны для какого-нибудь штамма (ЛФ82?)

In [4]:
grep "LF82" genomes_with_regions/pdu/*

genomes_with_regions/pdu/GCF_000284495.1_ASM28449v1_genomic.fna:>NC_011993.1 Escherichia coli LF82, complete genome


In [ ]:
ls /data11/bio/runs-manolov/from_7a/runs-manolov/ecoli_crohn/analyse/
